## Data Grouping for SIGNS Pipeline:
### The Francis Crick Institute, April 2021
#### This script will allow group variables that are extracted from images in CellProfiler by the tracked nucleus id.  
##### CellProfiler data is exported as a speadsheet with each nuclei in a seperate row for each image frame, thus if a nuclei is present in 10 images, it will have ten rows in the spreadsheet, each with a different id, but THE SAME tracked nucleus id.  This script groups by tracked nucleus id.

## Load the needed packages

In [ ]:
import pandas as pd
import os

### Specify the folder where the CellProfiler output files are located, and where to save the result

In [ ]:
#sel path is the path to the folder where the CellProfiler output files are
selpath = 'C:\\Users\\username\\Documents\\Projects\\Results from CellProfiler\\'

#change the filenames to the approrpriate file for your data
Filename_data_filename = 'MyExpt_Image.csv'  #makes a path for the csv file that has the filenames in it
Tracking_data_filename = 'MyExpt_Cells_Primary_objects.csv' #makes a path for the csv file that has the cell primary object data in it
Spots_data_filename = 'MyExpt_SpotsInCells_merged.csv' #makes a path for the csv file that has the merged spot data, including intesnity in it


#the Outpath is the path to the folder where you want to save the data
## Outfile_name is the filename you wish to use for the results.  Important! Keep the .xlsx extension for Excel
Outpath = 'C:\\Users\\username\\Documents\\Projects\\Results from CellProfiler\\'
Outfile_name = 'Output_Jupyter.xlsx'

In [ ]:
#look in the right path forthe data
Filename_data_filename = os.path.join(selpath,Filename_data_filename)  #makes a path for the csv file that has the filenames in it
Tracking_data_filename = os.path.join(selpath,Tracking_data_filename) #makes a path for the csv file that has the cell primary object data in it
Spots_data_filename = os.path.join(selpath,Spots_data_filename) #makes a path for the csv file that has the merged spot data, including intesnity in it


## Read in tables from the csv files

In [ ]:
## Read in tables from the csv files
Filename_table = pd.read_csv(Filename_data_filename, 'Delimiter', ',',skip_blank_lines=False) #read in the data for the filenames
Tracking_table = pd.read_csv(Tracking_data_filename,'Delimiter', ',',skip_blank_lines=False) #read in the data for the cell primary objects
Spots_table = pd.read_csv(Spots_data_filename); #read in the data for the merged spots and their intensities

In [ ]:
#Just view the filenames
Filename_table

# Construct a File_table dataframe

In [ ]:
#This is a bit redundant, but ported from Matlab, where it was used to convert table2array
#'ImageNumber'    #read in the image numbers from the Cell Profiler Spreadsheet. 
#'FileName_Cells' #read in the file name of the Cells Hu Channel from the CellProfiler Spreadsheet, 
#'FileName_Spots' #read in the file name of the RNAScope channel from the CellProfiler spreadsheet

data = {'ImageNumber':Filename_table.ImageNumber, 'CellsFile':Filename_table.FileName_Cells, 'SpotsFile':Filename_table.FileName_Spots}
File_table = pd.DataFrame(data, columns=['ImageNumber', 'CellsFile','SpotsFile'])
File_table

In [ ]:
Tracking_ids = Tracking_table.TrackObjects_Label_50
Tracking_ids = list(Tracking_ids.unique()) #get a list of the tracking IDs

## Define functions: 
### subset_by_track_id goes through the list of Tracking_ids and finds each image and object that correspond to that tracking id

### intensity_by_tracked_object sums all the spot intensity for the spots that are present in the objects found in subset_by_track_id for each tracking id

Note: each tracking ID corresponds to a single nucleus.

In [ ]:
def subset_by_track_id(Tracking_table, track_id):
    #Return a list of (Image Numbers, ObjectNumbers) that correspond to a specific track ID, for later summing
    print('Processing Tracked object:', track_id)
    
    SubTracking_Table=Tracking_table[Tracking_table.TrackObjects_Label_50==track_id] #Select out of the table only the rows for the tracked object
    ImageNumbers = list(SubTracking_Table['ImageNumber']) #get the image numbers where the tracked object occured
    ObjectNumbers = list(SubTracking_Table['ObjectNumber']) #get the object number in those image numbers
    IONumbers = list(zip(ImageNumbers, ObjectNumbers))
    
    return IONumbers

In [ ]:
def intensity_by_tracked_object(Spots_Table, IOList):
    ###Run through all the image planes and object for a single tracked object, and return the summed intensity
    Total_intensity=0
    for cell in IOList:
        Spots_Table_image = Spots_table[Spots_table.ImageNumber==cell[0]]
        
        if not Spots_Table_image[Spots_Table_image.ObjectNumber==cell[1]].shape[0]==0:
            Spots_Table_image_object = Spots_Table_image[Spots_Table_image.ObjectNumber==cell[1]]
            Intensity = float(Spots_Table_image_object.Intensity_IntegratedIntensity_Spots_eroded)
            Total_intensity = Total_intensity+Intensity
        
        else:
            Total_intensity = Total_intensity+0  #there's a quirk in how CellProfiler gave the data in the spreadsheet.  
                                                #it is possible for an object to exist and have spots in one image but another.
                                                #if that happens, there won't be a row present in the spots table where the 
                                                #image doesn't have any spots detected.  In the real world, this would correspond
                                                #to the top or bottom of a nucleus, where there might not be any spots detected.
                                                #This if-else checks if there are spots, and if not, adds zero.
    
    return Total_intensity

In [ ]:
Intensity_by_cell = []
cell_number = []
for j in Tracking_ids:
    IONumbers = subset_by_track_id(Tracking_table, j)
    
    Total_intensity = intensity_by_tracked_object(Spots_table, IONumbers)
    
    Intensity_by_cell.append(Total_intensity)
    cell_number.append(j)

In [ ]:
#Write out the results to the excel file specified in the beginning
export_data = {'ID':cell_number, 'Total_Intensity':Intensity_by_cell}
Export_File_table = pd.DataFrame(export_data, columns=['ID', 'Total_Intensity'])
Export_File_table.to_excel(os.path.join(Outpath, Outfile_name), index=False)
Export_File_table